# Modelos simples
O objetivo desse notebook é criar modelos que sejam simples para termos uma base de comparação no futuro.

In [2]:
# Importando bibliotecas
import fastf1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from typing import Optional

In [3]:
# importando dataset
df_2019 = pd.read_parquet('..\\Entregas_IA\\1_Bimestre\\f1_data-2019.parquet')
df_2019

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,session_type,Year,EventName,Round
0,0 days 00:38:08.968000,GAS,10,NaT,1.0,1.0,0 days 00:18:45.227000,0 days 00:20:41.628000,0 days 00:00:45.260000,0 days 00:00:29.297000,...,1,NaN,False,,False,False,Practice 1,2019,Australian Grand Prix,None
1,0 days 00:40:24.158000,GAS,10,0 days 00:02:15.190000,2.0,2.0,0 days 00:38:13.655000,NaT,0 days 00:00:59.328000,0 days 00:00:33.198000,...,1,NaN,False,,False,False,Practice 1,2019,Australian Grand Prix,None
2,0 days 00:41:52.770000,GAS,10,0 days 00:01:28.612000,3.0,2.0,NaT,NaT,0 days 00:00:29.417000,0 days 00:00:23.938000,...,1,NaN,False,,False,True,Practice 1,2019,Australian Grand Prix,None
3,0 days 00:43:46.351000,GAS,10,0 days 00:01:53.581000,4.0,2.0,NaT,NaT,0 days 00:00:39.078000,0 days 00:00:31.450000,...,1,NaN,False,,False,True,Practice 1,2019,Australian Grand Prix,None
4,0 days 00:45:12.775000,GAS,10,0 days 00:01:26.424000,5.0,2.0,NaT,NaT,0 days 00:00:28.624000,0 days 00:00:23.407000,...,1,NaN,False,,False,True,Practice 1,2019,Australian Grand Prix,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58942,0 days 01:46:33.783000,STR,18,0 days 00:01:46.966000,41.0,3.0,NaT,NaT,0 days 00:00:18.303000,0 days 00:00:45.415000,...,1,17.0,False,,False,True,Race,2019,Abu Dhabi Grand Prix,None
58943,0 days 01:48:19.324000,STR,18,0 days 00:01:45.541000,42.0,3.0,NaT,NaT,0 days 00:00:18.215000,0 days 00:00:44.856000,...,1,17.0,False,,False,True,Race,2019,Abu Dhabi Grand Prix,None
58944,0 days 01:50:05.042000,STR,18,0 days 00:01:45.718000,43.0,3.0,NaT,NaT,0 days 00:00:18.224000,0 days 00:00:44.842000,...,1,17.0,False,,False,True,Race,2019,Abu Dhabi Grand Prix,None
58945,0 days 01:51:53.842000,STR,18,0 days 00:01:48.800000,44.0,3.0,NaT,NaT,0 days 00:00:18.325000,0 days 00:00:45.586000,...,1,17.0,False,,False,True,Race,2019,Abu Dhabi Grand Prix,None


In [3]:
# print all object types
df_2019.dtypes

Time                  timedelta64[ns]
Driver                         object
DriverNumber                   object
LapTime               timedelta64[ns]
LapNumber                     float64
Stint                         float64
PitOutTime            timedelta64[ns]
PitInTime             timedelta64[ns]
Sector1Time           timedelta64[ns]
Sector2Time           timedelta64[ns]
Sector3Time           timedelta64[ns]
Sector1SessionTime    timedelta64[ns]
Sector2SessionTime    timedelta64[ns]
Sector3SessionTime    timedelta64[ns]
SpeedI1                       float64
SpeedI2                       float64
SpeedFL                       float64
SpeedST                       float64
IsPersonalBest                 object
Compound                       object
TyreLife                      float64
FreshTyre                        bool
Team                           object
LapStartTime          timedelta64[ns]
LapStartDate           datetime64[ns]
TrackStatus                    object
Position    

In [8]:
def get_dnfs_for_race(session) -> pd.DataFrame:
    """
    Returns a DataFrame of DNFs (excluding DSQ) for a single race in the given F1 season.

    DataFrame columns:
    - Driver: Full name of the driver
    - Circuit: Location of the circuit
    - Lap: Lap number at which the driver retired
    - Status: The retire reason/status from results
    - Weather columns: All available weather metrics at retirement time
    """
    # # Extract circuit info
    # event = fastf1.get_event_schedule(year)
    # circuit = event.loc[event['RoundNumber'] == round_number, 'Location'].iloc[0]

    # Identify DNFs by status (exclude 'Finished' and 'Disqualified')
    results = session.results
    retirements = results[
        (results['Status'] != 'Finished') &
        (results['Status'] != 'Disqualified') &
        ~(results['Status'].str.endswith('Lap')) &
        ~(results['Status'].str.endswith('Laps'))
        ][['Abbreviation', 'FullName', 'Status']]

    weather_cols = [c for c in session.weather_data.columns if c != 'Time']
    final_cols = ['Driver', 'Lap', 'Status'] + weather_cols

    if retirements.empty:
        # Create a row with None values for all columns
        no_dnf_data = {col: None for col in final_cols}
        return pd.DataFrame([no_dnf_data])

    # Determine last lap per retired driver
    laps = session.laps[['Driver', 'LapNumber', 'Time']]
    laps_ret = laps[laps['Driver'].isin(retirements['Abbreviation'])]
    last_idx = laps_ret.groupby('Driver')['LapNumber'].idxmax()
    last_laps = laps_ret.loc[last_idx].rename(
        columns={'Driver': 'Abbreviation', 'LapNumber': 'Lap', 'Time': 'RetireTime'}
    )

    # Merge with driver names and status
    merged = pd.merge(retirements, last_laps, on='Abbreviation')

    # Merge-as-of to align weather at retirement time
    weather_df = session.weather_data.sort_values('Time')
    merged = merged.sort_values('RetireTime')
    merged_weather = pd.merge_asof(
        merged,
        weather_df,
        left_on='RetireTime',
        right_on='Time',
        direction='backward'
    )

    # Assemble final DataFrame
    # merged_weather['Circuit'] = circuit
    # Drop helper cols
    merged_weather = merged_weather.drop(columns=['Abbreviation', 'RetireTime', 'Time']).rename(columns={'FullName': 'Driver'})
    # Reorder columns
    df = merged_weather[final_cols]

    return df

# example usage
# dnfs_br = get_dnfs_for_race(2023, 18)

In [8]:
# dnfs_br = pd.concat([
#     get_dnfs_for_race(2019, round_number)
#     for round_number in fastf1.get_event_schedule(2019)['RoundNumber']
# ], ignore_index=True)
# dnfs_br

core           INFO 	Loading data for Australian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 16: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '44', '33', '5', '16', '20', '27', '7', '18', '26', '10', '4', '11', '23', '99', '63', '88

,Driver,Circuit,Lap,Status,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,Carlos Sainz,Melbourne,10.0,Engine,23.5,70.3,1015.4,False,42.3,98,1.2
1,Daniel Ricciardo,Melbourne,29.0,Damage,23.1,71.0,1015.3,False,40.6,151,2.4
2,Romain Grosjean,Melbourne,30.0,Wheel,22.8,72.3,1015.3,False,40.0,187,1.1
3,Romain Grosjean,Sakhir,16.0,Retired,26.1,55.5,1003.7,False,28.7,34,2.4
4,Nico Hulkenberg,Sakhir,54.0,Engine,26.1,53.9,1003.3,False,27.6,52,2.6
5,Daniel Ricciardo,Sakhir,54.0,Out of fuel,26.1,54.0,1003.3,False,27.6,39,3.0
6,Carlos Sainz,Sakhir,53.0,Collision damage,26.1,53.8,1003.2,False,27.6,40,2.7
7,Nico Hulkenberg,Shanghai,16.0,Power Unit,19.6,43.3,1019.2,False,27.2,104,3.8
8,Daniil Kvyat,Shanghai,41.0,Collision,19.3,47.3,1019.3,False,25.7,89,4.2
9,Lando Norris,Shanghai,50.0,Collision,19.1,48.6,1019.3,False,25.4,80,3.7


In [19]:
# print(dnfs_br['Status'].unique())

['Engine' 'Damage' 'Wheel' 'Retired' 'Out of fuel' 'Collision damage'
 'Power Unit' 'Collision' 'Transmission' 'Brakes' 'Suspension' 'Spun off'
 'Exhaust' 'Power loss' 'Accident' 'Water pressure' 'Hydraulics'
 'Withdrew' 'Overheating' 'Oil leak']


In [19]:
def get_race_info(session) -> pd.DataFrame:
    """
    Fetches race info for the given Grand Prix city and year.

    Returns a DataFrame with:
      - nr_corners: Number of corners on the circuit
      - laps: Number of laps in the race
      - total_distance_km: Total race distance in kilometers
      - start_air_temp_c: Air temperature at race start
      - start_track_temp_c: Track temperature at race start
      - start_relative_humidity: Relative humidity at race start
      - elevation_change_m: Total elevation change over a representative lap (m)
    """

    # Circuit info
    ci = session.get_circuit_info()
    try:
        nr_corners: int = list(ci.corners.Number)[-1]
    except IndexError:
        nr_corners: int = int(input())

    # Laps and total distance
    laps: int = session.total_laps
    # lap_distance_km: float = session.event.circuit.length_km
    # total_distance_km: float = lap_distance_km * laps

    # Weather: take the first recorded sample as race start
    weather_df = session.weather_data
    if not weather_df.empty:
        w0 = weather_df.iloc[0]
        start_air_temp_c = w0.get('AirTemp')
        start_track_temp_c = w0.get('TrackTemp')
        start_relative_humidity = w0.get('Humidity', None)
        start_rainfall = w0.get('Rainfall', None)
        start_wind_speed = w0.get('WindSpeed', None)
        start_wind_direction = w0.get('WindDirection', None)
    else:
        start_air_temp_c = start_track_temp_c = start_relative_humidity = start_rainfall = start_wind_speed = start_wind_direction = None

    # Elevation change: use the winner's fastest lap telemetry (Z channel)
    # results = session.results
    # winner_code = results.loc[results['Position'] == 1, 'Driver'].iloc[0]
    fastest_lap = session.laps.pick_fastest()
    telemetry = fastest_lap.get_car_data()
    if 'Z' in telemetry:
        z = telemetry['Z']
        elevation_change_m = float(z.max() - z.min())
    else:
        elevation_change_m = None

    # Build summary DataFrame
    summary = pd.DataFrame([{
        'Nr_corners': nr_corners,
        'Laps': laps,
        # 'TotalDistance_km': total_distance_km,
        'StartAirTemp_c': start_air_temp_c,
        'StartTrackTemp_c': start_track_temp_c,
        'StartRelativeHumidity': start_relative_humidity,
        'StartRainfall': start_rainfall,
        'StartWindSpeed': start_wind_speed,
        'StartWindDirection': start_wind_direction,
        'ElevationChange_m': elevation_change_m
    }])

    return summary

# circuit_info = get_race_info("São Paulo", 2023)
# circuit_info

In [13]:
# circuit_info.dtypes

Nr_corners                 int64
Laps                       int64
StartAirTemp_c           float64
StartTrackTemp_c         float64
StartRelativeHumidity    float64
StartRainfall               bool
StartWindSpeed           float64
StartWindDirection         int64
ElevationChange_m         object
dtype: object

In [17]:
STATUS_CATEGORY = {
    # Track-related
    "Accident": "hazard",
    "Collision": "hazard",
    "Spun off": "hazard",
    "Collision damage": "hazard",
    # Mechanical-but-not-track
    "Engine": "mechanical",
    "Wheel": "mechanical",
    "Power Unit": "mechanical",
    "Transmission": "mechanical",
    "Brakes": "mechanical",
    "Suspension": "mechanical",
    "Exhaust": "mechanical",
    "Oil leak": "mechanical",
    "Power loss": "mechanical",
    "Overheating": "mechanical",
    # Other
    "Electrical": "other",
    "Hydraulics": "other",
    "Water pressure": "other",
    "Out of fuel": "other",
    "Retired": "other",
    "Withdrew": "other",
    "Damage": "other"
}

def engineer_dnf_features(dnfs: pd.DataFrame) -> pd.DataFrame:
    # check the dnfs DataFrame for NaN values except for the 'Circuit' column
    if dnfs[dnfs.columns.difference(['Circuit'])].isna().all().all() or dnfs.shape[0] == 0:
        # If all values are NaN or empty, return an empty DataFrame
        dummy_data = {
            'Circuit': ['DummyCircuit'],
            'Driver': ['DummyDriver'],
            'Status': ['Accident'],  # Assuming this would map in STATUS_CATEGORY
            'Lap': [1],
            'AirTemp': [25.0],
            'Humidity': [50.0],
            'Pressure': [1013.0],
            'Rainfall': ['No'],
            'TrackTemp': [30.0],
            'WindSpeed': [10.0]
        }

        dummy_df = pd.DataFrame(dummy_data)

        dummy_result = engineer_dnf_features(dummy_df)

        # Create the result with None values but same structure
        result = pd.DataFrame({col: [None] for col in dummy_result.columns})

        return result

# 1. Clean
    dnfs = dnfs.copy()
    dnfs["Rainfall_bool"] = dnfs["Rainfall"].map({"Yes": 1, "No": 0}).fillna(0)
    # 2. Map status to coarse categories
    dnfs["status_cat"] = dnfs["Status"].map(STATUS_CATEGORY).fillna("other")

    # 3. Aggregate per circuit
    aggs = {
        "Driver":      "count",          # total DNFs
        "Lap":         ["mean", "std"],
        "AirTemp":     "mean",
        "Humidity":    "mean",
        "Pressure":    "mean",
        "Rainfall_bool": "mean",
        "TrackTemp":   "mean",
        "WindSpeed":   "mean",
    }
    grouped = dnfs.groupby("Circuit").agg(aggs)
    # flatten columns
    grouped.columns = [
        "_".join(col).strip() for col in grouped.columns.to_flat_index()
    ]
    # rename
    grouped = grouped.rename(columns={"Driver_count": "total_dnfs"})
    # rate per lap
    total_laps = dnfs.groupby("Circuit")["Lap"].sum().rename("sum_laps")
    grouped = grouped.join(total_laps)
    grouped["dnf_rate_per_lap"] = grouped["total_dnfs"] / grouped["sum_laps"]

    status_pct = (
        dnfs
        .pivot_table(index="Circuit",
                     columns="status_cat",
                     values="Driver",
                     aggfunc="count",
                     fill_value=0)
    )
    # Calculate row sums first
    row_sums = status_pct.sum(axis=1)
    # Then divide each value by its row sum to get percentages
    status_pct = status_pct.div(row_sums, axis=0).add_prefix("pct_")

    return grouped.join(status_pct).reset_index()


def merge_with_static(dnf_feats: pd.DataFrame, static_info: pd.DataFrame) -> pd.DataFrame:
    if static_info.shape[0] != 1 or dnf_feats.shape[0] != 1:
        raise ValueError("Both inputs must be single-row DataFrames.")
    s = static_info.reset_index(drop=True)
    d = dnf_feats.reset_index(drop=True)
    merged = pd.concat([s, d], axis=1)
    for col in d.columns:
        merged[col] = merged[col].fillna(0)

    return merged


def pipeline_dataset(start_year: int, end_year: int) -> pd.DataFrame:
    # Input validation
    if not isinstance(start_year, int) or not isinstance(end_year, int):
        raise TypeError("Both start_year and end_year must be integers")

    if start_year > end_year:
        raise ValueError("start_year must be less than or equal to end_year")

    # Initialize an empty DataFrame to store the results
    dataframe = pd.DataFrame()

    # Iterate through each year in the range
    for year in range(start_year, end_year + 1):
        # try:
            # Get the schedule for the current year
            schedule = fastf1.get_event_schedule(year, include_testing=False)
            # Extract track names using vectorized operation and update the set
            track_names = schedule['EventName']
            for track_name in track_names:
                session = fastf1.get_session(year, track_name, 'R')
                session.load(laps=True, weather=True, telemetry=True)
                race_info = get_race_info(session)
                dnfs = get_dnfs_for_race(session)
                dnfs["Circuit"] = track_name
                dnf_features = engineer_dnf_features(dnfs)
                features = merge_with_static(dnf_features, race_info)
                dataframe = pd.concat([dataframe, features], ignore_index=True)
        # except Exception as e:
        #     print(f"Error processing year {year}: {str(e)}")
        #     continue
            print(f"Year {year} processed successfully.")

    # Reset index
    dataframe.reset_index(drop=True, inplace=True)
    return dataframe

In [21]:
# Test the pipeline
df = pipeline_dataset(2019, 2020)
df

core           INFO 	Loading data for Australian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 16: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '44', '33', '5', '16', '20', '27', '7', '18', '26', '10', '4', '11', '23', '99', '63', '88

Year 2019 processed successfully.


core           INFO 	Loading data for Austrian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2020/1/results.json failed; using cached response
Traceback (most recent call last):
  File "C:\Users\rafam\Projects\race-engineer\.venv\Lib\site-packages\requests_cache\session.py", line 286, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rafam\Projects\race-engineer\.venv\Lib\site-packages\requests_cache\session.py", line 254, in _send_and_cache
    response = super().send(request, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rafam\Projects\race-engineer\.venv\Lib\site-packages\fastf1\req.py", line 134, in send
    lim.limit()
  File "C:\Users\rafam\Projects\race-engineer\.venv\Lib\site-packa

KeyboardInterrupt: 